## Partie I: Preparation des données (preprocessing)

In [1]:
import pandas as pd
import requests
from io import BytesIO #permet de stocker en mémoire
from zipfile import ZipFile
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

url2019 = "https://www.insee.fr/fr/statistiques/fichier/4809583/fd_eec19_csv.zip" #enquete 2019
url2020="https://www.insee.fr/fr/statistiques/fichier/5393560/fd_eec20_csv.zip" #enquête 2020 en exemple

# Télécharge le fichier ZIP
requete = requests.get(url2019)
zip_df = ZipFile(BytesIO(requete.content)) #créer un fichier ZIP

# Extraire le fichier CSV du ZIP
with zip_df.open(zip_df.namelist()[0]) as extrait:
    EEC_2019 = pd.read_csv(extrait, delimiter=";") # Lire le fichier CSV avec pandas

# Télécharge le fichier ZIP
requete = requests.get(url2020)
zip_df = ZipFile(BytesIO(requete.content)) #créer un fichier ZIP

# Extraire le fichier CSV du ZIP
with zip_df.open(zip_df.namelist()[0]) as extrait:
    EEC_2020 = pd.read_csv(extrait, delimiter=";") # Lire le fichier CSV avec pandas

In [2]:
# On choisit un set de variables qui ne comprend que données sociodémographiques, géographiques associées à l'individu
# et au ménage étudié + qui soit disponible sur les deux années étudiées

# Var d'interet -> ACTEU

# Var explicatives  à inclure dans le modèle -> AGE3 ;  AGE5 ; ANNEE ;TRIM ;  CATAU2010R ; COURED ; CSTOT (pour avoir actifs et inactifs) ;CSTOTR ;
# CSTOTPRM ; DIP11; ENFRED ; METRODOM ; NFRRED ; SEXE ; TYPMEN7 

# USE? -> AIDFAM , CHPUB?? (Employeur de la profession principale?), EXTRIAN? (pondération)

# Meme si les variables sont colinéaires (AGE3, AGE5), on peut les garder et faire en sorte que le predicteur
# choisisse la plus pertinente

# Je pense que le problème principal vient du fait qu'on a en fait trop peu de variable, et donc rien qui ne puisse être vraiment pertinent en fait
#c'est pour cela que les algorithmes ne trouvent rien je pense. (Alexis)

list_var_selected = ["ACTEU","ANNEE" ,"TRIM", "AGE3" ,  "AGE5"  , "CATAU2010R" ,
"COURED" ,"CSTOT" ,"CSTOTR" ,"CSTOTPRM" , "DIP11","ENFRED" , "METRODOM" , "NFRRED" , "SEXE" , "TYPMEN7"]

EEC_2019 = EEC_2019[list_var_selected]
EEC_2020 = EEC_2020[list_var_selected]

list_var = list(EEC_2019.columns.values)
print(list_var)

#je regarde ce qu'il se passe si je ne sépare pas les trimestres, le prof avait l'air de dire que c'était pas forcément grave (Alexis)
#EEC_2019 = EEC_2019[EEC_2019['TRIM'] ==1]
#EEC_2020 = EEC_2020[EEC_2020['TRIM'] ==4]

# Il y a très peu de valeurs manquantes dans les variables ->  dropna() drops any row that contains at least one missing value
# On aurait aussi pu faire de l'imputation
print(EEC_2019.shape)
EEC_2019 = EEC_2019.dropna() 
EEC_2020 = EEC_2020.dropna() 
print(EEC_2019.shape)

# Converti l'ensemble des variables catégorielles en dummies -> Réflèxe économétrique mais est ce pertinent ici? 
# Oui oui faut le faire, par contre pour les variables qui sont DEJA des dummies, c'est pas la peine, faut juste les recoder en 0 et 1
EEC_2019 = pd.get_dummies(EEC_2019, columns=["AGE3" ,  "AGE5"  , "CATAU2010R" ,
"CSTOT" ,"CSTOTR" ,"CSTOTPRM" , "DIP11", "NFRRED" , "TYPMEN7"])
EEC_2020 = pd.get_dummies(EEC_2020, columns=["AGE3" ,  "AGE5"  , "CATAU2010R" ,
"CSTOT" ,"CSTOTR" ,"CSTOTPRM" , "DIP11", "NFRRED" , "TYPMEN7"])

#du coup je recode les variables en questions
EEC_2019['FEMME'] = EEC_2019['SEXE'] - 1
EEC_2020['FEMME'] = EEC_2020['SEXE'] - 1

EEC_2019['COUPLE'] = 2 - EEC_2019['COURED']
EEC_2020['COUPLE'] = 2 - EEC_2020['COURED']

EEC_2019['ENFANT'] = 2 - EEC_2019['ENFRED']
EEC_2020['ENFANT'] = 2 - EEC_2020['ENFRED']

EEC_2019['DOM'] = EEC_2019['METRODOM'] - 1
EEC_2020['DOM'] = EEC_2020['METRODOM'] - 1

#et je vire les anciennes
EEC_2019 = EEC_2019.drop(['METRODOM', 'ENFRED' , 'COURED', 'SEXE'], axis=1)
EEC_2020 = EEC_2020.drop(['METRODOM', 'ENFRED' , 'COURED', 'SEXE'], axis=1)

# 133 variables du coup
print(EEC_2019.shape)

# Gestion des outliers??? Pas nécessaire ici puisqu'on fonctionne seulement avec des variables catégorielles

#si besoin de travailler sur subsample
#EEC_2019_subsample = EEC_2019.sample(n=1000, random_state=3) 


['ACTEU', 'ANNEE', 'TRIM', 'AGE3', 'AGE5', 'CATAU2010R', 'COURED', 'CSTOT', 'CSTOTR', 'CSTOTPRM', 'DIP11', 'ENFRED', 'METRODOM', 'NFRRED', 'SEXE', 'TYPMEN7']
(416298, 16)
(412028, 16)
(412028, 133)


In [3]:
# On construit un array contentant l'ensemble des variables explicatives (on exclut donc ACTEU/ANNEE/TRIM)
X = np.array(EEC_2019.drop(columns=["ACTEU","ANNEE", "TRIM"]))

print(X.shape)
# Array contentant la variable expliquée
y = np.array(EEC_2019["ACTEU"])
# convert "Chomage" in 1 and the other labels (here, "Inactif", "Actif occupé") into 1
y = np.where(y==2, 1, 0)
print(y.shape)

print(X[:10])

(412028, 130)
(412028,)
[[False False True ... 0 0 0]
 [False False True ... 1 0 0]
 [False False True ... 1 0 0]
 ...
 [False True False ... 1 1 0]
 [False True False ... 1 1 0]
 [True False False ... 0 0 0]]


In [4]:
#from sklearn.preprocessing import StandardScaler
# standardisation des variables explicatives,  c’est-à-dire de centrer et réduire chaque variable
# en la divisant par sa variance  -> pas utile pour mes variables catégorielles transformées en dummies

# std_scale = StandardScaler().fit(X_train)
# X_train_scaled = std_scale.transform(X_train)
# X_test_scaled = std_scale.transform(X_test)

# La standardisation permet une convergence plus rapide des algorithmes pour des variables réelles


## Partie II : Support Vector Machine

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=.2, random_state=3)

In [ ]:
#SVC est le modèle SVM permettant la classification
#Test avec nombreuses valeurs paramètre C en vue de réalisation d'un graphique

acc_train, acc_test = list(), list()

C_range = np.linspace(0.1, 20, 50)
for param in C_range:
    clf = SVC( C=param)
    clf.fit(X_train, y_train)
    #on garde les scores pour le graph
    acc_train.append(clf.score(X_train, y_train))
    acc_test.append(clf.score(X_test, y_test))
    #et on fait les confusions matrix
    y_pred = clf.predict(X_test)
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title(f"Confusion Matrix for C={param}")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()
      
#  ATTENTION -> Peut etre faut-il travailler avec un autre critère de performance que accuracy pour selectionner
#  le meilleur modèle


In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(C_range, acc_train, label='train set', lw=4)
plt.plot(C_range, acc_test, label='test set', lw=4)

plt.legend(loc='best', fontsize=12)

plt.xlabel("C", fontweight="bold", fontsize=20)
plt.ylabel("Performance", fontweight="bold", fontsize=20)
plt.xticks(fontweight="bold", fontsize=15)
plt.yticks(fontweight="bold", fontsize=15)
plt.tight_layout()

plt.show()